In [6]:
import numpy as np
from reservoirpy.nodes import Reservoir, Ridge, Input
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

# 1단계: 데이터 전처리
# 예시 한글 리뷰 데이터 (여기에 실제 리뷰 데이터를 넣으세요)
reviews = [
    "이 제품 정말 좋아요!",
    "별로에요, 다시는 사고 싶지 않아요.",
    "정말 최고예요, 추천합니다!",
    "가격 대비 실망스러워요.",
    "완전 대박이에요, 강력 추천합니다!"
]

# 해당 감정 레이블 (1은 긍정, 0은 부정)
labels = [1, 0, 1, 0, 1]




In [7]:
# 텍스트 데이터를 TF-IDF로 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews).toarray()  # 텍스트를 수치 벡터로 변환
y = np.array(labels)

# 2단계: 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# 3단계: 저수지 컴퓨팅 모델 생성
input_dim = X_train.shape[1]  # 입력 데이터의 차원 (특징의 수)
reservoir = Reservoir(100, lr=0.3, sr=1.1)  # 100개의 유닛을 가진 저수지
readout = Ridge(output_dim=1, ridge=1e-6)  # 읽기 아웃 계층

# 모델 파이프라인 구성
esn = Input(input_dim=input_dim) >> reservoir >> readout


In [11]:
# 4단계: 모델 학습
esn.fit(X_train, y_train.reshape(-1, 1))  # y_train의 형태를 (N, 1)로 맞춰줍니다.

Running Model-3: 4it [00:00, 3998.38it/s]             
Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 83.34it/s]

Fitting node Ridge-1...


'Model-3': Model('Input-1', 'Reservoir-1', 'Ridge-1')

In [12]:
# 5단계: 모델 테스트
predictions = esn.run(X_test)
predictions = predictions > 0.5  # 연속적인 출력을 0 또는 1로 이진화

# 6단계: 성능 평가
accuracy = accuracy_score(y_test, predictions)
print(f"정확도: {accuracy * 100:.2f}%")

Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 996.04it/s]

정확도: 100.00%
